In [1]:
import pandas as pd
from sqlalchemy import create_engine
from config.config import conn #connection to SQL database (build on a RP4)
from dash import Input, Output, html, dcc, dash
import plotly.graph_objects as go

In [97]:
#connect to database
def conn():
    covid_all = pd.read_sql('SELECT * FROM covid_data', conn)


#load data locally
covid_all = pd.read_csv('owid-covid-data.csv')

#Transform the date
covid_all['date'] = pd.to_datetime(covid_all['date'])

In [98]:
df = covid_all[['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases', 'total_deaths', 'new_deaths']]

In [125]:
df['continent'].fillna(df['location'])
continents = df['continent'].unique()
continents.sort()

In [99]:
countries = df['location'].unique()
countries.sort()

agrupations = ['World', 'High income', 'Upper middle income', 'Asia', 'Europe', 'North America', 'Lower middle income', 'United States', 'European Union', 'South America']

In [102]:
external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css']
tickFont = {'size':12, 'color':"rgb(30,30,30)",
            'family':"Courier New, monospace"}

In [131]:
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.layout = html.Div(
    style={ 'font-family':"Courier New, monospace" }, children=[
        html.H1('Case History of the Coronavirus (COVID-19)'),
        html.Div(className="row", children=[
            html.Div(className="four columns", children=[
                html.H5('Country'),
                dcc.Dropdown( 
                    id='country',
                    options=[{'label':c, 'value':c} for c in countries],
                    value='Spain'
                )
            ]),
            html.Div(className="four columns", children=[
                html.H5('Continent'),
                dcc.Dropdown(
                    id='continent',
                    options=[{'label': c, 'value': c} for c in continents],
                    value='Europe'
                )
            ]),
            html.Div(className="four columns", children=[
                html.H5('Selected Metrics'),
                dcc.Checklist(
                    id='metrics',
                    options=[{'label':m, 'value':m} for m in ['New Cases', 'New Deaths']],
                    value=['new_cases', 'new_deaths']
                )
            ])
        ]),
    
        dcc.Graph(
            id="plot_new_metrics",
            config={ 'displayModeBar': False }
        ),
        dcc.Graph(
            id="plot_cum_metrics", 
            config={ 'displayModeBar': False }
    )    
    ]
)

In [111]:
df.loc[df['location'] == 'Spain'].drop('location', axis=1)

,iso_code,continent,date,total_cases,new_cases,total_deaths,new_deaths
111181,ESP,Europe,2020-02-01,1.0,1.0,NaN,NaN
111182,ESP,Europe,2020-02-02,1.0,0.0,NaN,NaN
111183,ESP,Europe,2020-02-03,1.0,0.0,NaN,NaN
111184,ESP,Europe,2020-02-04,1.0,0.0,NaN,NaN
111185,ESP,Europe,2020-02-05,1.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...
111823,ESP,Europe,2021-11-04,5022546.0,3291.0,87477.0,15.0
111824,ESP,Europe,2021-11-05,5025639.0,3093.0,87504.0,27.0
111825,ESP,Europe,2021-11-06,5025639.0,0.0,87504.0,0.0
111826,ESP,Europe,2021-11-07,5025639.0,0.0,87504.0,0.0


In [132]:
def nonreactive_data(country):
    data = df.loc[df['location'] == country] \
                  .drop(['location', 'iso_code', 'continent'], axis=1)
    data['dateStr'] = data['date'].dt.strftime('%b %d, %Y')
    return data

In [134]:
colors = {'Deaths':'rgb(200,30,30)',
          'Confirmed':'rgb(100,140,240)'}

def barchart(data, metrics, prefix="", yaxisTitle=""):
    figure = go.Figure(data=[
        go.Bar(
            name=metric, x=data.date, y=data[prefix + metric],
            marker_line_color='rgb(0,0,0)', marker_line_width=1,
            marker_color=colors[metric]
        ) for metric in metrics
    ])
    figure.update_layout( 
              barmode='group', legend=dict(x=.05, y=0.95), 
              plot_bgcolor='#FFFFFF', font=tickFont) \
          .update_xaxes( 
              title="", tickangle=-90, type='category',  
              showgrid=True, gridcolor='#DDDDDD', 
              tickfont=tickFont, ticktext=data.dateStr,                     
              tickvals=data.date) \
          .update_yaxes(
              title=yaxisTitle, showgrid=True, gridcolor='#DDDDDD')
    return figure

In [137]:
@app.callback(
    Output('plot_new_metrics', 'figure'), 
    [Input('country', 'value'), Input('continent', 'value'), 
     Input('metrics', 'value')]
)
def update_plot_new_metrics(country, metrics):
    data = nonreactive_data(country)
    return barchart(data, metrics, prefix="New", 
        yaxisTitle="New Cases per Day")
@app.callback(
    Output('plot_cum_metrics', 'figure'), 
    [Input('country', 'value'), Input('continent', 'value'), 
     Input('metrics', 'value')]
)
def update_plot_cum_metrics(country, metrics):
    data = nonreactive_data(country)
    return barchart(data, metrics, prefix="Cum", 
        yaxisTitle="Cumulated Cases")